### Extract nutrients from Nutritionix (https://www.nutritionix.com/) by API calling
##### Authors: Reshma


* Instacart dataset :
    * Instacart product file - 36000 unique products which inlcude non food items too
    
* API ID and API Key:
    * We have contacted 50+ people to get API IDs and API keys which was used to extract nutrient informtaions about instacart food products.
    

In [ ]:
import pandas as pd
import numpy as np
from nutritionix import Nutritionix
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import defaultdict

In [13]:
#Load instacart datasets
dataPath = 'C:/Users/SSK/Documents/UC - MScA/Courses/Capstone/Datasets/'
savePath = 'C:/Users/SSK/Documents/UC - MScA/Courses/Capstone/Code/insta_scrape/'
#load API key and IDs saved in nutri_apis file
apis     = pd.read_csv(dataPath+'nutri_apis.csv')
df       = pd.read_csv(dataPath+"instacart_online_grocery_shopping_2017_05_01/james_3000.csv")

In [44]:
apis

,Name,API ID,API Key
41,Roy Xie,3c9c0e47,086db97209fb70fb8aa90fe563c9b33f
42,Li Cao,00021e9f,b2e3493833812aa45a438ed104634be9
43,Xiao Li,b4e69d6d,533cdde457ce889a2566b0dd1b5beaaa
44,Elisha Yu,7861c9c9,75024367ab872932b588c5bbea0d618f
45,JY Yu,4242ca05,f5f3aebe345b4c4046435dec858190ca
46,Harper Xiang,49ddd24e,4df0fc7fc8fdab4b114ce6f120e74579
47,Han Huang,2260100d,d69b85179d55dce55e7f49fdd6debcd1
48,Mark Wang,904723a0,ac59dd4edfa203bac6f9b2978b30242f
49,Eason Li,9f9b0c59,fc8f32e8866e5b4e780046cb1ae7ba8b
50,Terry Li,2bfb8e16,2f347d137b96f003735652f953b739ac


In [27]:
df['0']     = df['product_name'].apply(lambda x : x.replace('/', ' '))
df['0']     = df['0'].apply(lambda x : x.replace('\\', ''))
df['0']     = df['0'].apply(lambda x : x.replace(',', ''))
df['0']     = df['0'].apply(lambda x : x.replace('california', ''))
df['0']     = df['0'].apply(lambda x : x.replace('  ', ' '))
df['0']     = df['0'].apply(lambda x : x.replace('(', ''))
df['0']     = df['0'].apply(lambda x : x.replace(')', ''))
df['0']     = df['0'].apply(lambda x :x.replace('\"', ' inch'))
df['0']     = df['0'].apply(lambda x :x.replace('+', ' plus'))
df['0']     = df['0'].apply(lambda x :x.replace('%', ' percent'))

product_uni = df['0'].values

In [48]:
lastdaycount = n #manual put number due to daily limit on extraction
counter      = lastdaycount+1
notfound     = ()

In [50]:
for row in apis.iterrows():
    rowv = row[1].values
    up_limit = counter
    lw_limit = counter+50
    print ('processing..', rowv[0], up_limit, lw_limit)
    nix     = Nutritionix(app_id=rowv[1].strip(), api_key=rowv[2].strip())
    df1     = pd.DataFrame()
    prods   =  product_uni[up_limit:lw_limit]
    for idx, prod in enumerate (prods):
        prod_search = nix.search(prod)
        results     = prod_search.json()
        prod_dict = defaultdict(list)
        print ("Processing:{}, index:{}".format(prod, idx))
        if 'hits' in results.keys():
            if len(results['hits'])>0:
                for hit in results['hits']:
                    hit_prod = hit['fields']['item_name']
                    prod_dict['item_id'].append(hit['fields']['item_id'])
                    prod_dict['item_name'].append(hit_prod)
                    prod_dict['item_score'].append(fuzz.token_set_ratio(hit_prod, prods))

                prod_df = pd.DataFrame(prod_dict)
                prod_df_sorted = prod_df.sort_values(by =['item_score'] , ascending=False)
                best_scr  = prod_df_sorted['item_score'].values[0]
                res_df   = prod_df_sorted[prod_df_sorted['item_score'] == best_scr] 
                res_df.sort_index(inplace=True)
                sel_item_id = res_df.head(1)['item_id'].values[0]
                
                scrape_dict_1 = nix.item(id=sel_item_id).json()
                scrape_dict_1['CH_Product_Name'] = prod
                print (scrape_dict_1)
                df1 = pd.concat([df1, pd.DataFrame.from_dict(scrape_dict_1,orient='index').T])
            else:
                print ('Not found products:', prod)
            
    f_lw_limit = lw_limit -1
    filename = 'date_insta'+str(up_limit)+'_'+str(f_lw_limit)+'.csv'
    df1.to_csv(savePath+'todayScrape/'+filename)
    counter = lw_limit

processing.. Roy Xie 2030 2080
Processing:baked crackers cheddar cheese, index:0
{'old_api_id': None, 'item_id': '5489b90aa051f633376f9314', 'item_name': 'Cheddar Cheese Baked Crackers', 'leg_loc_id': None, 'brand_id': '51db3825176fe9790a89b756', 'brand_name': 'Miltons', 'item_description': None, 'updated_at': '2016-10-16T04:36:15.000Z', 'nf_ingredient_statement': 'Brown Rice Flour, Corn Flour, Gluten Free Oats, Potato Starch, High Oleic Sunflower Oil (Tocopherols, Rosemary Extract and Ascorbyl Palmitate Added As An Antioxidant), Potato Flakes, Cheddar Flavor (Tapioca Dextrin, Salt, Sunflower Oil, Natural Flavor, Silicon Dioxide), Organic Cheese Blend (Organic Cheddar Cheese [Organic Cultured Pasteurized Milk, Salt, Enzymes], Organic Nonfat Milk, Organic Buttermilk, Salt, Disodium Phosphate), Long Grain White Rice Flour, Molasses Granules (Syrup, Molasses), Cane Sugar, Yeast (Sorbitan Monostearate, Ascorbic Acid), Sea Salt, Baking Soda, Monocalcium Phosphate, Sunflower Lecithin, Sodium

Processing:cheese sausage and pepperoni topped mini bagels, index:6
{'old_api_id': None, 'item_id': '5792ffd1deabd5d3316b83ac', 'item_name': 'Mini Bagels, Cheese, Sausage & Pepperoni', 'leg_loc_id': None, 'brand_id': '51db37d9176fe9790a89a005', 'brand_name': 'Price Rite', 'item_description': None, 'updated_at': '2016-07-23T05:30:41.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 190, 'nf_calories_from_fat': 50, 'nf_total_fat': 6, 'nf_saturated_fat': 2.5, 'nf_trans_fatty_acid': 0, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 15, 'nf_sodium': 440, 'nf_total_carbohydrate': 27, 'nf_dietary_fiber': 2, 'nf_sugars': 3, 'nf_protein': 9, 'nf_vitamin_a_dv': 0, 'nf_vitamin_c_dv': 8, 'nf_calcium_dv': 10, 'nf_iron_dv': 10, 'nf_refuse_pct': None, 'nf_servings_per_container': 2, 'nf_serving_size_qty': 4, 'nf_serving_size_unit': 'pieces', 'nf_serving_weight_grams': 88, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'a

Processing:mushroom spinach and salsa tamales, index:12
{'old_api_id': None, 'item_id': '5aed57978529caba38b60014', 'item_name': 'Mushroom, Spinach & Salsa Tamales', 'leg_loc_id': None, 'brand_id': '5268310b3203b9c3390000e2', 'brand_name': 'Primavera', 'item_description': None, 'updated_at': '2018-05-05T07:05:01.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 250, 'nf_calories_from_fat': 100, 'nf_total_fat': 11, 'nf_saturated_fat': 1.5, 'nf_trans_fatty_acid': None, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 0, 'nf_sodium': 260, 'nf_total_carbohydrate': 37, 'nf_dietary_fiber': 6, 'nf_sugars': 3, 'nf_protein': 5, 'nf_vitamin_a_dv': None, 'nf_vitamin_c_dv': None, 'nf_calcium_dv': None, 'nf_iron_dv': None, 'nf_refuse_pct': None, 'nf_servings_per_container': None, 'nf_serving_size_qty': 1, 'nf_serving_size_unit': 'serving', 'nf_serving_weight_grams': None, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'a

Processing:calabrese uncured italian salami with pepper and paprika, index:18
{'old_api_id': None, 'item_id': '545ce785d43cefdf6dbf8087', 'item_name': 'Calabrese Italian Salami', 'leg_loc_id': None, 'brand_id': '545ce7b24ce901f7437d9ed4', 'brand_name': 'Creminelli', 'item_description': None, 'updated_at': '2014-11-24T20:24:24.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 110, 'nf_calories_from_fat': 7, 'nf_total_fat': 8, 'nf_saturated_fat': 3, 'nf_trans_fatty_acid': 0, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 30, 'nf_sodium': 470, 'nf_total_carbohydrate': 0, 'nf_dietary_fiber': 0, 'nf_sugars': 0, 'nf_protein': 8, 'nf_vitamin_a_dv': 0, 'nf_vitamin_c_dv': 0, 'nf_calcium_dv': 0, 'nf_iron_dv': 2, 'nf_refuse_pct': None, 'nf_servings_per_container': 23, 'nf_serving_size_qty': 1, 'nf_serving_size_unit': 'oz', 'nf_serving_weight_grams': 28, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'allergen_contain

Processing:couscous organic whole wheat, index:24
{'old_api_id': None, 'item_id': '5c6babf0b59a3f7e65591e4e', 'item_name': 'Organic Whole Wheat Couscous', 'leg_loc_id': None, 'brand_id': '526831133203b9c3390001c8', 'brand_name': 'Unknown', 'item_description': None, 'updated_at': '2019-02-19T07:10:44.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 105, 'nf_calories_from_fat': None, 'nf_total_fat': 0, 'nf_saturated_fat': 0, 'nf_trans_fatty_acid': 0, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 0, 'nf_sodium': 3, 'nf_total_carbohydrate': 22, 'nf_dietary_fiber': 1, 'nf_sugars': 0, 'nf_protein': 4, 'nf_vitamin_a_dv': 0, 'nf_vitamin_c_dv': 0, 'nf_calcium_dv': 1, 'nf_iron_dv': 0, 'nf_refuse_pct': None, 'nf_servings_per_container': None, 'nf_serving_size_qty': 28, 'nf_serving_size_unit': 'g', 'nf_serving_weight_grams': 28, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'allergen_contains_fish': None, 'allergen

...

Processing:condensed milk sweetened full cream, index:44
{'old_api_id': None, 'item_id': '5e54c67ad50f4a3c4fc95467', 'item_name': 'Condensed Milk, Full Cream Sweetened', 'leg_loc_id': None, 'brand_id': '54fdf47c3b05d44e59f1c2f9', 'brand_name': 'Sun Sweet', 'item_description': None, 'updated_at': '2020-02-28T07:04:32.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 130, 'nf_calories_from_fat': 30, 'nf_total_fat': 3, 'nf_saturated_fat': 2, 'nf_trans_fatty_acid': 0, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 10, 'nf_sodium': 0.04, 'nf_total_carbohydrate': 22, 'nf_dietary_fiber': 0, 'nf_sugars': 22, 'nf_protein': 3, 'nf_vitamin_a_dv': 2, 'nf_vitamin_c_dv': 0, 'nf_calcium_dv': 10, 'nf_iron_dv': 0, 'nf_refuse_pct': None, 'nf_servings_per_container': 10, 'nf_serving_size_qty': 2, 'nf_serving_size_unit': 'tbsp', 'nf_serving_weight_grams': None, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'allergen_contains

Processing:eggs cheese & turkey sausage breakfast burrito, index:0
{'old_api_id': None, 'item_id': '5892daee42e46a672e3656e4', 'item_name': 'Breakfast Burrito, Turkey Sausage Egg & Cheese', 'leg_loc_id': None, 'brand_id': '51db380f176fe9790a89b1ce', 'brand_name': "Red's", 'item_description': None, 'updated_at': '2019-03-02T11:02:59.000Z', 'nf_ingredient_statement': 'turkey, salt.', 'nf_water_grams': None, 'nf_calories': 310, 'nf_calories_from_fat': 0, 'nf_total_fat': 13, 'nf_saturated_fat': 5, 'nf_trans_fatty_acid': 0, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 115, 'nf_sodium': 560, 'nf_total_carbohydrate': 29, 'nf_dietary_fiber': 1, 'nf_sugars': 2, 'nf_protein': 19, 'nf_vitamin_a_dv': 8, 'nf_vitamin_c_dv': 10, 'nf_calcium_dv': 20, 'nf_iron_dv': 15, 'nf_refuse_pct': None, 'nf_servings_per_container': 10, 'nf_serving_size_qty': 1, 'nf_serving_size_unit': 'burrito', 'nf_serving_weight_grams': 142, 'allergen_contains_milk': None, 'allergen_contains_

Processing:organic thyme, index:6
{'old_api_id': None, 'item_id': '5eccc0c1b1a3fa3d0af8b117', 'item_name': 'Organic Lemon Thyme', 'leg_loc_id': None, 'brand_id': '526830fb3203b9c339000046', 'brand_name': 'Whole Foods Market', 'item_description': None, 'updated_at': '2020-05-26T07:09:58.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 140, 'nf_calories_from_fat': None, 'nf_total_fat': 0, 'nf_saturated_fat': None, 'nf_trans_fatty_acid': None, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': None, 'nf_sodium': 45, 'nf_total_carbohydrate': 36, 'nf_dietary_fiber': None, 'nf_sugars': 35, 'nf_protein': 0, 'nf_vitamin_a_dv': None, 'nf_vitamin_c_dv': None, 'nf_calcium_dv': 0, 'nf_iron_dv': 0, 'nf_refuse_pct': None, 'nf_servings_per_container': 2, 'nf_serving_size_qty': 12, 'nf_serving_size_unit': 'fl oz', 'nf_serving_weight_grams': None, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'allergen_contains_fish': None, 

Processing:olives organic kalamata pitted, index:8
{'old_api_id': None, 'item_id': '51c373bb97c3e69de4b0c139', 'item_name': 'Organic Kalamata Pitted Olives', 'leg_loc_id': None, 'brand_id': '51db37d7176fe9790a899f99', 'brand_name': 'Peloponnese', 'item_description': 'Organic Kalamata, Pitted', 'updated_at': '2014-11-24T20:24:24.000Z', 'nf_ingredient_statement': 'Organic Kalamata Olives (Organically Grown and Processed in Accordance with the USDA NDP standards), Water, Organic Red Wine Vinegar, Sea Salt. Gluten free.', 'nf_water_grams': None, 'nf_calories': 45, 'nf_calories_from_fat': 40, 'nf_total_fat': 4.5, 'nf_saturated_fat': 0.5, 'nf_trans_fatty_acid': None, 'nf_polyunsaturated_fat': None, 'nf_monounsaturated_fat': None, 'nf_cholesterol': 0, 'nf_sodium': 210, 'nf_total_carbohydrate': 1, 'nf_dietary_fiber': None, 'nf_sugars': None, 'nf_protein': 0, 'nf_vitamin_a_dv': None, 'nf_vitamin_c_dv': None, 'nf_calcium_dv': None, 'nf_iron_dv': None, 'nf_refuse_pct': None, 'nf_servings_per_cont

Processing:sesame oil refined, index:15
{'old_api_id': None, 'item_id': '5f08199dd9d8e9493562047f', 'item_name': 'Refined Sesame Oil', 'leg_loc_id': None, 'brand_id': '552aff292a2736f104190307', 'brand_name': 'Signature Select', 'item_description': None, 'updated_at': '2020-07-10T07:32:49.000Z', 'nf_ingredient_statement': None, 'nf_water_grams': None, 'nf_calories': 130, 'nf_calories_from_fat': 126, 'nf_total_fat': 14, 'nf_saturated_fat': 2, 'nf_trans_fatty_acid': None, 'nf_polyunsaturated_fat': 6, 'nf_monounsaturated_fat': 6, 'nf_cholesterol': 0, 'nf_sodium': 0, 'nf_total_carbohydrate': 0, 'nf_dietary_fiber': None, 'nf_sugars': None, 'nf_protein': 0, 'nf_vitamin_a_dv': None, 'nf_vitamin_c_dv': None, 'nf_calcium_dv': None, 'nf_iron_dv': None, 'nf_refuse_pct': None, 'nf_servings_per_container': 33, 'nf_serving_size_qty': 1, 'nf_serving_size_unit': 'tbsp', 'nf_serving_weight_grams': None, 'allergen_contains_milk': None, 'allergen_contains_eggs': None, 'allergen_contains_fish': None, 'all